In [1]:
from ptyrodactyl import simul, tools, workflows
import jax
import jax.numpy as jnp

W1205 17:44:24.589807  297651 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1205 17:44:24.594193  297650 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1205 17:44:24.598436  297647 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1205 17:44:24.598963  297645 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1205 17:44:24.600034  297648 cuda_executor.cc:1802] GPU interconnect information not available: INTERNAL: NVML doesn't support extracting fabric info or NVLink is not used by the device.
W1205 17:44:24.635221  297644 cuda_executor.cc:1802] GPU int

In [2]:
jax.devices()

[CudaDevice(id=0),
 CudaDevice(id=1),
 CudaDevice(id=2),
 CudaDevice(id=3),
 CudaDevice(id=4),
 CudaDevice(id=5),
 CudaDevice(id=6),
 CudaDevice(id=7)]

In [3]:
mos2_gb_file = "../../data/for_CBED_MoS2_poly.xyz"

In [4]:
mos2_data: tools.CrystalData = simul.parse_crystal(mos2_gb_file)
mos2_data.positions.shape

(21600, 3)

In [5]:
x_pos = mos2_data.positions[:, 0]
y_pos = mos2_data.positions[:, 1]
z_pos = mos2_data.positions[:, 2]

In [6]:
jnp.max(x_pos), jnp.min(x_pos), jnp.max(y_pos), jnp.min(y_pos), jnp.max(z_pos), jnp.min(z_pos)


(Array(116.73292915, dtype=float64),
 Array(-78.2934247, dtype=float64),
 Array(101.25578357, dtype=float64),
 Array(-67.78546024, dtype=float64),
 Array(11.47243469, dtype=float64),
 Array(1.00274303, dtype=float64))

In [7]:
voltage_kv = 60
cbed_aperture_mrad = 5.0
cbed_extent_mrad = 50.0
cbed_shape = (256, 256)
real_space_pixel_size_ang = 0.1
slice_thickness = 1.0


In [8]:
yy, xx = jnp.meshgrid(jnp.linspace(-50, 100, 150), jnp.linspace(-50, 100, 150))
scan_positions = jnp.asarray((yy.ravel(), xx.ravel())).T
scan_positions

Array([[-50.        , -50.        ],
       [-48.99328859, -50.        ],
       [-47.98657718, -50.        ],
       ...,
       [ 97.98657718, 100.        ],
       [ 98.99328859, 100.        ],
       [100.        , 100.        ]], dtype=float64)

In [9]:
cbed_simulation = workflows.crystal2stem4d(
    crystal_data=mos2_data, 
    scan_positions=scan_positions, 
    voltage_kv=voltage_kv, 
    cbed_aperture_mrad=cbed_aperture_mrad, 
    cbed_extent_mrad=cbed_extent_mrad, 
    cbed_shape=cbed_shape, 
    real_space_pixel_size_ang=real_space_pixel_size_ang, 
    slice_thickness=slice_thickness,
    force_parallel=True
    )

E1205 17:44:30.133007  297022 gpu_hlo_schedule.cc:817] The byte size of input/output arguments (1294942270424) exceeds the base limit (12526534656). This indicates an error in the calculation!
W1205 17:44:30.134410  297022 hlo_rematerialization.cc:3204] Can't reduce memory use below 1.18TiB (1294884360000 bytes) by rematerialization; only reduced to 2.35TiB (2589826660800 bytes), down from 2.35TiB (2589826660832 bytes) originally
W1205 17:44:40.345416  297022 bfc_allocator.cc:501] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.18TiB (rounded to 1294884360192)requested by op 
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
W1205 17:44:40.345635  297022 bfc_allocator.cc:512] **__________________________________________________________________________________________________
E1205 17:44:40.345656  297022 pjrt_stream_exe

JaxRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 1.18TiB.